# LastFM Popularity

<b>What</b> are the most popular music genres in LastFM? <b>Who</b> are the most popular artists in each genre, and <b>where</b> do they come from? <br>
<b>When</b> the music was made? <b>What</b> are their mood?

These are the questions that these project makes. We will levarege the [dataset](https://www.kaggle.com/pieca111/music-artists-popularity) made available in Kaggle by [Piotr](https://www.kaggle.com/pieca111) to answer them.

### Lets take a look at the dataset.

In [127]:
import pandas as pd

df = pd.read_csv(
        'Datasets/artists.csv', # https://www.kaggle.com/pieca111/music-artists-popularity
        dtype=dict(
            artist_lastfm=str, 
            country_lastfm=str, 
            tags_lastfm=str,   
            )
        )
        
df.head()

,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


To answer the questions we posed at the beginning, we will need:<br>
- <b>ambiguous_artist</b> | Sometimes more than one artist may share the same name
- <b>artist_lastfm</b> | Artist name in LastFM
- <b>country_lastfm</b> | Their countries
- <b>tags_lastfm</b> | Their tags
- <b>listeners_lastfm</b> | You get the idea.
- <b>scrobbles_lastfm</b>




In [128]:
df = df[['ambiguous_artist', 'artist_lastfm', 'country_lastfm', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm']]
df.head()

,ambiguous_artist,artist_lastfm,country_lastfm,tags_lastfm,listeners_lastfm,scrobbles_lastfm
0,False,Coldplay,United Kingdom,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0
1,False,Radiohead,United Kingdom,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0
2,False,Red Hot Chili Peppers,United States,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0
3,False,Rihanna,Barbados; United States,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0
4,False,Eminem,United States,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0


Way better.
### <b>However</b>

In [129]:
df.tail()

,ambiguous_artist,artist_lastfm,country_lastfm,tags_lastfm,listeners_lastfm,scrobbles_lastfm
1466078,False,NaN,South Korea,NaN,NaN,NaN
1466079,False,NaN,NaN,NaN,NaN,NaN
1466080,False,NaN,NaN,NaN,NaN,NaN
1466081,False,NaN,South Korea,NaN,NaN,NaN
1466082,False,NaN,South Korea,NaN,NaN,NaN


Looks like we have our fair share of null values

In [130]:
null_rate = (df.isna().sum() / len(df)) * 100

ax = (
    null_rate
    .sort_values()
    .plot.barh(
        title='Percentage of null values by column',
        figsize=(20,8)
        )
    )

The column with the highest number of null values is country_lastfm, with a <b>null rate of approximately 85%</b>. <br>

It might reasonable to assume that the artists with no country filled in their lastfm page should not be among the most popular. Lets drop all null values and check how many plays we lose

In [131]:
print('Total scrobbles (plays) before filtering null values:')
print(f'{df["scrobbles_lastfm"].sum():,}')

df.dropna(axis=0, how='any', inplace=True)

print('...and after:')
print(f'{df["scrobbles_lastfm"].sum():,}')

Total scrobbles (plays) before filtering null values:
120,324,972,038.0
...and after:
102,827,413,080.0


So we are losing less than 20% of the total plays by doing so.<br><br>
Now, lets see how much we lose by dropping the ambiguous artists

In [132]:
df.drop(df[df['ambiguous_artist']].index, inplace=True)
df.drop(df[df['scrobbles_lastfm'] <= 0].index, inplace=True) # lets drop artists with no plays as well

print('Total number of plays:' ,f'{df["scrobbles_lastfm"].sum():,}')
print('Total number of unique artists:' ,f'{len(df):,}')

Total number of plays: 87,214,810,301.0
Total number of unique artists: 171,128


So we are set with 3/4 of the original number of plays and with over 170 thousand unique artists. That will do. <br><br>
Some standard dataprep:

In [133]:
df.drop(['ambiguous_artist'], axis=1, inplace=True) # We don't need this column anymore
df.columns = ['artist', 'country', 'tags','listeners', 'scrobbles'] # If not at the sacrifice of clarity, shorter names = good
df[['scrobbles', 'listeners']] = df[['scrobbles', 'listeners']].astype(int) # There are no half listeners (nor half scrobble)
df.sort_values(by='scrobbles', ascending=False, inplace=True)

df.head()

,artist,country,tags,listeners,scrobbles
17,The Beatles,United Kingdom,classic rock; rock; british; 60s; pop,3674017,517126254
1,Radiohead,United Kingdom,alternative; alternative rock; rock; indie; el...,4732528,499548797
0,Coldplay,United Kingdom,rock; alternative; britpop; alternative rock; ...,5381567,360111850
8,Muse,United Kingdom,alternative rock; rock; alternative; Progressi...,4089612,344838631
23,Arctic Monkeys,United Kingdom,indie rock; indie; british; rock; alternative;...,3501680,332306552


### We could really use these tags, if only they were each in their own row....

In [134]:
df['tags'] = df['tags'].apply(lambda x: x.split('; '))
df['country'] = df['country'].apply(lambda x: x.split('; ')) # Some groups/bands have members from multiple countries

df = df.explode('tags')
df = df.explode('country')

df.reset_index(inplace=True, drop=True)
df.head()

,artist,country,tags,listeners,scrobbles
0,The Beatles,United Kingdom,classic rock,3674017,517126254
1,The Beatles,United Kingdom,rock,3674017,517126254
2,The Beatles,United Kingdom,british,3674017,517126254
3,The Beatles,United Kingdom,60s,3674017,517126254
4,The Beatles,United Kingdom,pop,3674017,517126254


### There you go.

In [135]:
tags = df.groupby(by='tags').size().sort_values(ascending=False)
print(f'{len(tags):,}')

323,038


We have <b>323,038</b> unique tags.

#### <b>Which is quite a lot.</b> <br>
<br>
Lets take a look at them

In [136]:
print(tags.index[:50])

Index(['seen live', 'rock', 'electronic', 'pop', 'All', 'indie', 'alternative',
       'female vocalists', 'under 2000 listeners', 'experimental', 'american',
       'folk', 'metal', 'punk', 'male vocalists', 'electronica', 'ambient',
       'spotify', 'USA', 'jazz', 'singer-songwriter', 'instrumental',
       'hardcore', '00s', 'dance', 'alternative rock', 'indie rock', 'german',
       'japanese', 'british', 'Hip-Hop', 'punk rock', '90s', 'psychedelic',
       'electro', 'world', '80s', 'chillout', 'death metal', 'acoustic',
       'french', 'black metal', 'rap', 'indie pop', 'female vocalist',
       'hard rock', 'hip hop', 'techno', 'heavy metal', 'soul'],
      dtype='object', name='tags')


To reach our stated goals, we will need to go through the fisrt x tags and classify them as one of the following:
<br>
- Musical genre
- Mood
<br>
<br>
I've wrote a small script to make it a little less tedious to do so (check the file tag_flagger.py if you are interested).
<br>
Due to this unortodox approach I ended up with the tags classification spread across a few files. That's why I'm reading them with this loop. 

In [137]:
import os

tags_csv_path = f'Datasets\\tags_classification\\'
tags_csv_dir = os.listdir(tags_csv_path)
df_tags = pd.DataFrame(columns=['tag_category', 'tags'])

for file in tags_csv_dir:
    if '.csv' in file:
        df_temp = pd.read_csv(tags_csv_path + file, delimiter=';', index_col=0, names=['tag_category', 'tags'])
        df_tags = pd.concat([df_tags, df_temp], ignore_index=True)

df_tags.head()

,tag_category,tags
0,genre,metal
1,genre,punk
2,genre,jazz
3,genre,hardcore
4,genre,alternative rock


### Was this genre more active in the 60s or the 70s?
The decade tags are easy to antecipate, so we will type them down.

In [138]:
df_decades = pd.DataFrame(data={'tags': ['40s', '50s', '60s', '70s', '80s', '90s', '00s', '10s']})
df_decades['tag_category'] = 'decade'
df_tags = df_tags.append(df_decades)

df_tags.tail()

C:\Users\thari\AppData\Local\Temp\ipykernel_2856\775998077.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tags = df_tags.append(df_decades)


,tag_category,tags
3,decade,70s
4,decade,80s
5,decade,90s
6,decade,00s
7,decade,10s


### It looks like some tags can be written in more than one way...

In [139]:
df_tags[(df_tags['tags'] == "rock'n'roll") | (df_tags['tags'] == 'rock n roll')]

,tag_category,tags
75,genre,rock'n'roll
184,genre,rock n roll


In this case if we removed empty spaces and the ' character we would be able to map both instanced of Rock'n'Roll tag to the same value

In [140]:
df_tags['tag_clean'] = df_tags['tags'].str.upper()

(df_tags['tag_clean']
    .replace(['&', 'AND'], 'N',
    regex=True, inplace=True))

(df_tags['tag_clean']
    .replace(["'", ' ', '-'], '',
    regex=True, inplace=True))
    
df_tags['tag_clean'][:20].values

array(['METAL', 'PUNK', 'JAZZ', 'HARDCORE', 'ALTERNATIVEROCK',
       'INDIEROCK', 'PUNKROCK', 'ELECTRO', 'CHILLOUT', 'DEATHMETAL',
       'BLACKMETAL', 'RAP', 'INDIEPOP', 'HARDROCK', 'HIPHOP', 'TECHNO',
       'HEAVYMETAL', 'SOUL', 'HOUSE', 'PROGRESSIVEROCK'], dtype=object)

How many duplicate tags were we able to map?

In [141]:
print('Unique tags before cleaning:', len(df_tags))
print('Unique tags after cleaning: ', len(df_tags.drop_duplicates(subset='tag_clean')))

Unique tags before cleaning: 249
Unique tags after cleaning:  219


So we got rid of 30 duplicate tags<br>
We will match all variations of a given genre to the way it was written in its first appearence

In [142]:
tags_to_keep_series = df_tags.groupby('tag_clean')['tags'].first()
df_tags['tag_to_keep'] = df_tags['tag_clean'].map(tags_to_keep_series)
df_tags[df_tags['tag_clean'] == 'ROCKNROLL']

,tag_category,tags,tag_clean,tag_to_keep
75,genre,rock'n'roll,ROCKNROLL,rock'n'roll
184,genre,rock n roll,ROCKNROLL,rock'n'roll
190,genre,Rock and Roll,ROCKNROLL,rock'n'roll


Its time to bring out tags selection, mapping and classification to the main dataframe  

In [143]:
# I'm not using inner join to filter because it sorts the dataset and we will leverage the tag order soon
df = df.merge(df_tags[['tags', 'tag_to_keep', 'tag_category']],
            on='tags', how='left')
df.dropna(subset=['tag_to_keep'], inplace=True)

df.drop(columns='tags', inplace=True)

df = df[['artist', 'country', 'tag_category', 'tag_to_keep', 'listeners', 'scrobbles']]
df.columns = ['artist', 'country', 'tag_category', 'tags', 'listeners', 'scrobbles']

df['tags'] = df['tags'].map(dict(melancholy='melancholic')).fillna(df['tags'])

We don't need two tags telling me that Led Zeppelin is Rock'n'Roll

In [144]:
df[(df['tags'] == "rock'n'roll") & (df['artist'] == 'Led Zeppelin')]

,artist,country,tag_category,tags,listeners,scrobbles
900,Led Zeppelin,United Kingdom,genre,rock'n'roll,2930957,188306100
907,Led Zeppelin,United Kingdom,genre,rock'n'roll,2930957,188306100


In [145]:
df.drop_duplicates(subset=['artist', 'country', 'tags'], inplace=True)
df[(df['tags'] == "rock'n'roll") & (df['artist'] == 'Led Zeppelin')]

,artist,country,tag_category,tags,listeners,scrobbles
900,Led Zeppelin,United Kingdom,genre,rock'n'roll,2930957,188306100


### But...
We still have an issue to solve.<br>
Lets take a look at all the genres that we are currently assigning to <b>Radiohead<b/>

In [146]:
df[(df['artist']=='Radiohead') & (df['tag_category']=='genre')][['artist', 'tags']]

,artist,tags
6,Radiohead,alternative rock
12,Radiohead,britpop
13,Radiohead,indie rock
16,Radiohead,Progressive rock
20,Radiohead,Experimental Rock
21,Radiohead,post-rock
24,Radiohead,pop
34,Radiohead,chillout
37,Radiohead,indie pop
38,Radiohead,trip-hop


They might have a few songs that can be understood as chillout, but this is by no means their genre. So as to avoid having Radiohead as the most listened chillout artist, we will select only the <b>top 3 genres per artist</b>.
<br>
<br>
I'm not sure how lastfm ranks them, but they are certainly ranked by relevance (probably by the number of users that set each tag to an artist)



In [148]:
df_genres = df[df['tag_category'] == 'genre']
df['tag_order'] = df_genres.groupby('artist').cumcount() + 1
df['tag_order'] = df['tag_order'].fillna(1).astype(int)
df = df[df['tag_order'] < 4]
df.drop(columns='tag_order', inplace=True)

df['tags'] = df['tags'].str.title()

df[(df['artist']=='Radiohead') & (df['tag_category']=='genre')][['artist', 'tags']]



,artist,tags
6,Radiohead,Alternative Rock
12,Radiohead,Britpop
13,Radiohead,Indie Rock


Way better. <br>

Our main analysis will be based around the genres. So we will break down our table in three:
<br>
<br>
![Tab](resources/tables_relationship.png)
<br>
<br>
Where the genres table will keep all the attributes.

In [150]:
def df_slice_by_category(df, tag_category: str, cols: list):
    # Slices the dataframe based on the tag category

    filter = df['tag_category'] == tag_category
    df_slice = df.loc[filter, cols]
    df_slice.drop_duplicates(inplace=True)

    df_slice.rename(
        columns=dict(tags=tag_category), 
        inplace=True
        )
    
    return df_slice

df_decades = df_slice_by_category(df, tag_category='decade', cols=['artist', 'tags', 'scrobbles'])
df_moods = df_slice_by_category(df, tag_category='mood', cols=['artist', 'tags', 'scrobbles'])
df_genres = df_slice_by_category(df, tag_category='genre', cols=['artist', 'tags'])

In out moods and decades tables we will have the following structure:
<br>
<br>
![Mood Fields](resources/moods_fields.png)
<br>
<br>
Essentially, a rank of mood/decade by genre

In [151]:
def rank_by_genre(df_genres, df_tag, tag_category, filter=False, threshold=15):

    df_tag = df_tag.merge(df_genres, on='artist', how='inner')
    df_tag.drop_duplicates(inplace=True)

    df_tag = (
        df_tag.groupby(['genre', tag_category])
        .sum()
        .reset_index()
        .sort_values(
            by=['genre', 'scrobbles'], 
            ascending=False
            )
        )

    if filter:

        df_tag['rank'] = df_tag.groupby('genre').cumcount()
        filter = df_tag['rank'] < threshold
        df_tag = df_tag.loc[filter, df_tag.columns[:-1]]

    return df_tag

In [93]:
df_decades = rank_by_genre(df_genres, df_decades, tag_category='decade')
df_decades.to_csv('Datasets/decades.csv', index=False)

I originally had the idea of <b>assigning colors to the different moods (e.g. melancholy = deep blue)</b> and use that to change the whole color scheme of the dashboard to match the most common mood for the selected genre.
<br>
Needless to say, to get all the different color schemes looking good was much more effort than I had set aside for this project. I'm keeping the related logic here in case I decide to implement this in the future.

In [ ]:
df_mood = rank_by_genre(df_genres, df_moods, tag_category='mood', filter=True)

df_mood_colors = pd.read_csv('Datasets\\mood_color.csv')
df_mood_colors.dropna(how='any', inplace=True)
df_mood_colors['mood'] = df_mood_colors['mood'].str.title()
df_mood_colors.head()

,mood,color
0,Trippy,purple
1,Spiritual,blue_light
3,Sexy,red
4,Sad,grey
5,Romantic,red


In [100]:
df_mood = df_mood.merge(df_mood_colors, on='mood', how='inner')
df_mood.to_csv('Datasets/moods.csv', index=False)

df_mood.head()

,genre,mood,scrobbles,color
0,Trip-Hop,Sexy,652016514.0,red
1,Trap,Sexy,440750.0,red
2,Trance,Sexy,34042170.0,red
3,Thrash Metal,Sexy,3042683.0,red
4,Teen Pop,Sexy,228185785.0,red


### Genres
Bringing the moods and colors to the genres dataset. 

In [152]:

genre_filter = df['tag_category'] == 'genre'
df = df.loc[genre_filter, df.columns]

df.rename(columns=dict(tags='genre'), inplace=True)
df.drop(columns='tag_category', inplace=True)
df = df.merge(df_mood[['genre', 'color', 'mood']], on='genre', how='left')
df.head(10)

,artist,country,genre,listeners,scrobbles,color,mood
0,The Beatles,United Kingdom,Classic Rock,3674017,517126254,red,Sexy
1,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_light,Mellow
2,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_dark,Melancholic
3,The Beatles,United Kingdom,Classic Rock,3674017,517126254,grey,Sad
4,The Beatles,United Kingdom,Classic Rock,3674017,517126254,yellow,Funky
5,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_dark,Cool
6,The Beatles,United Kingdom,Classic Rock,3674017,517126254,yellow,Comedy
7,The Beatles,United Kingdom,Classic Rock,3674017,517126254,orange,Fun
8,The Beatles,United Kingdom,Classic Rock,3674017,517126254,red,Romantic
9,The Beatles,United Kingdom,Classic Rock,3674017,517126254,orange,Happy


### But we need to be wary...<br> 
...of duplicating values when aggregating. <br>

Each artist appears in more than one row:

In [153]:
df.head(2)

,artist,country,genre,listeners,scrobbles,color,mood
0,The Beatles,United Kingdom,Classic Rock,3674017,517126254,red,Sexy
1,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_light,Mellow


So if we sum over an artist we end up multiplying its listeners and play count by the number of rows that it appears.

In [154]:
df.groupby('artist').sum().loc['The Beatles']

listeners    7.715436e+07
scrobbles    1.085965e+10
Name: The Beatles, dtype: float64

### We can handle that in Tableau... <br>
... by using aggregations such as "MAX". But that will require <b>Tableau to calculate it over and over again</b>.<br>
We can lend a hand to Tableau by creating boolean columns identifying unique rows.<br> <br>
We have increased the number of rows in our dataframe two times:

- When we expanded the <b>tags</b>
- and when we expanded the <b>countries</b>  <br>

We will start by creating a row count for each <b>artist</b> and each combination of <b>artist-tag</b> and <b>artist-country</b>

In [155]:
df['artist_unique'] =  df.groupby(['artist']).cumcount() + 1
df['genre_artist_unique'] = df.groupby(['artist', 'genre']).cumcount() + 1
df['country_artist_unique'] = df.groupby(['artist', 'country']).cumcount() + 1
df.head()

,artist,country,genre,listeners,scrobbles,color,mood,artist_unique,genre_artist_unique,country_artist_unique
0,The Beatles,United Kingdom,Classic Rock,3674017,517126254,red,Sexy,1,1,1
1,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_light,Mellow,2,2,2
2,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_dark,Melancholic,3,3,3
3,The Beatles,United Kingdom,Classic Rock,3674017,517126254,grey,Sad,4,4,4
4,The Beatles,United Kingdom,Classic Rock,3674017,517126254,yellow,Funky,5,5,5


And then flagging as <b>True</b> where the value is one

In [156]:
cols_to_bool = ['artist_unique', 'genre_artist_unique', 'country_artist_unique']

for col in cols_to_bool:
    df[col] = df[col].apply(lambda x: x==1)
    
df.head()

,artist,country,genre,listeners,scrobbles,color,mood,artist_unique,genre_artist_unique,country_artist_unique
0,The Beatles,United Kingdom,Classic Rock,3674017,517126254,red,Sexy,True,True,True
1,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_light,Mellow,False,False,False
2,The Beatles,United Kingdom,Classic Rock,3674017,517126254,blue_dark,Melancholic,False,False,False
3,The Beatles,United Kingdom,Classic Rock,3674017,517126254,grey,Sad,False,False,False
4,The Beatles,United Kingdom,Classic Rock,3674017,517126254,yellow,Funky,False,False,False


### It would be handy to have an artist rank by tag already in place

In [157]:
df.sort_values(by=['genre', 'scrobbles'], ascending=False, inplace=True)
df['artist_rank'] = df.groupby(by=['genre'])['scrobbles'].rank(method='dense', ascending=False).astype(int)

# Top 5 artists in rock'n'roll
df[df['genre'] == "Rock'N'Roll"][['artist', 'scrobbles', 'artist_rank']][:5]

,artist,scrobbles,artist_rank
4735,Elvis Presley,71070448,1
4736,Elvis Presley,71070448,1
4737,Elvis Presley,71070448,1
4738,Elvis Presley,71070448,1
4739,Elvis Presley,71070448,1


### Since we are at it, why not a country rank?

In [158]:
df_country = df[['genre', 'country', 'scrobbles']].groupby(by=['genre', 'country']).sum()
df_country.reset_index(inplace=True)
df_country.sort_values(by=['genre', 'scrobbles'], ascending=False, inplace=True)
df_country['country_rank'] = df_country.groupby(['genre'])['scrobbles'].rank(method='dense', ascending=False).astype(int)


df_country.head()

,genre,country,scrobbles,country_rank
11083,Trip-Hop,United Kingdom,9.358371e+09,1
11084,Trip-Hop,United States,5.721943e+09,2
11023,Trip-Hop,France,1.451972e+09,3
11031,Trip-Hop,Iceland,1.299964e+09,4
11059,Trip-Hop,Norway,8.806872e+08,5


In [159]:
df_country.drop(columns='scrobbles', inplace=True)
df = df.merge(df_country, on=['genre', 'country'], how='left')
df.head()

,artist,country,genre,listeners,scrobbles,color,mood,artist_unique,genre_artist_unique,country_artist_unique,artist_rank,country_rank
0,Lana Del Rey,United States,Trip-Hop,1881065,217157209,red,Sexy,False,True,False,1,2
1,Lana Del Rey,United States,Trip-Hop,1881065,217157209,blue_light,Mellow,False,False,False,1,2
2,Lana Del Rey,United States,Trip-Hop,1881065,217157209,blue_dark,Melancholic,False,False,False,1,2
3,Lana Del Rey,United States,Trip-Hop,1881065,217157209,purple_light,Ethereal,False,False,False,1,2
4,Lana Del Rey,United States,Trip-Hop,1881065,217157209,grey,Sad,False,False,False,1,2


# That's all


In [100]:
df.to_csv('Datasets/lastfm_genres.csv', index=False)